In [1]:
from selenium import webdriver
from time import sleep

driver = webdriver.Chrome('/Users/iliasmiraoui/Desktop/chromedriver')

In [238]:

artist= "Coldplay"
url = "https://www.riaa.com/gold-platinum/?tab_active=default-award&ar="+artist+"&lab=&genre=&format=&date_option=release&from=&to=&award=&type=&category=&adv=SEARCH#search_section"

In [283]:
driver.get(url)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51504): Max retries exceeded with url: /session/38bbee87c37962ffa2f907a75b41a839/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x110fcae90>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [245]:
artists = []
songs = []
last_certification = []
label = []
frmat= []
details = []
nb_songs = len(driver.find_elements_by_xpath("//*[@class='table_award_row']"))
if len(driver.find_element_by_xpath("//*[@id='loadmore']"))>0:
    driver.find_element_by_xpath("//*[@id='loadmore']").click()
to_click = driver.find_elements_by_xpath("//*[@class='others_cell format_cell']/div/a")
for link in to_click:
    link.click()
    #sleep(0.3)
awards = driver.find_elements_by_xpath("//*[@class='table_award_row expanded']")
for award in awards:
    artists.append(award.find_element_by_xpath(".//*[@class='artists_cell']").text)
    songs.append(award.find_element_by_xpath(".//*[@class='others_cell']").text)
    last_certification.append(award.find_elements_by_xpath(".//*[@class='others_cell']")[1].text)
    label.append(award.find_elements_by_xpath(".//*[@class='others_cell']")[2].text)
    frmat.append(award.find_element_by_xpath(".//*[@class='others_cell format_cell']").text)
    next_element = award.find_element_by_xpath(".//following-sibling::tr")
    detail = next_element.find_element_by_xpath("//*[@class='table_award_row expanded']")
    details.append([x.text for x in next_element.find_elements_by_xpath(".//*[@class='content_recent_table']/td")])

['SOMETHING JUST LIKE THIS', 'A SKY FULL OF STARS', 'HYMN FOR THE WEEKEND', 'ADVENTURE OF A LIFETIME', 'A HEAD FULL OF DREAMS', 'GHOST STORIES', 'MAGIC', 'PARADISE', 'MYLO XYLOTO', 'VIVA LA VIDA', 'VIVA LA VIDA', 'FIX YOU', 'X&Y', 'A RUSH OF BLOOD TO THE HEAD', 'YELLOW', 'THE SCIENTIST', 'SPEED OF SOUND', 'CLOCKS', 'COLDPLAY LIVE 2003', 'PARACHUTES', 'COLDPLAY LIVE 2003']
['SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'VIDEO LONGFORM\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'ALBUM\nMORE DETAILS']
[['FEBRUARY 17, 2017', '6x Multi-Platinum | November 8, 2019', 'None', 'Digital', '6 Million', 'POP', ' ', '5x Multi-Pla

In [6]:
def get_riaa(artist):
    query = artist
    query = urllib.parse.quote_plus(query)
    df = pd.DataFrame(columns=["artist","artists","songs","last_certification","label","format","details"])
    url = "https://www.riaa.com/gold-platinum/?tab_active=default-award&ar="+query+"&lab=&genre=&format=&date_option=release&from=&to=&award=&type=&category=&adv=SEARCH#search_section"
    driver.get(url)
    sleep(0.1)
    artists = []
    songs = []
    last_certification = []
    label = []
    frmat= []
    details = []
    hascontent =0
    try:
        driver.find_element_by_xpath("//*[@id='loadmore']").click()
    except:
        hascontent =1
        sleep(0.3)
    to_click = driver.find_elements_by_xpath("//*[@class='others_cell format_cell']/div/a")
    for link in to_click:
        link.click()
        sleep(0.6)
    awards = driver.find_elements_by_xpath("//*[@class='table_award_row expanded']")
    if hascontent ==1:
        awards.extend(driver.find_elements_by_xpath("//*[@class='table_award_row expanded hasContent']"))
    nb_songs = len(awards)
    for award in awards:
        artists.append(award.find_element_by_xpath(".//*[@class='artists_cell']").text)
        songs.append(award.find_element_by_xpath(".//*[@class='others_cell']").text)
        last_certification.append(award.find_elements_by_xpath(".//*[@class='others_cell']")[1].text)
        label.append(award.find_elements_by_xpath(".//*[@class='others_cell']")[2].text)
        frmat.append(award.find_element_by_xpath(".//*[@class='others_cell format_cell']").text)
        next_element = award.find_element_by_xpath(".//following-sibling::tr")
        detail = next_element.find_element_by_xpath("//*[@class='table_award_row expanded']")
        details.append([x.text for x in next_element.find_elements_by_xpath(".//*[@class='content_recent_table']/td")])
    for i in range(nb_songs):
        df.loc[i] = [artist,artists[i],songs[i],last_certification[i],label[i],frmat[i],details[i]]
    return df

In [8]:
import pandas as pd
import urllib.parse
import pickle 
with open('artist_df.pkl', 'rb') as f:
        artist_df = pickle.load(f)
with open('song_df_100.pkl', 'rb') as f:
        song_df = pickle.load(f)


In [ ]:
from tqdm import tqdm_notebook as tqdm
driver = webdriver.Chrome('/Users/iliasmiraoui/Desktop/chromedriver')
pbar = tqdm(song_df["artist"].unique())
i = 0
for artist in pbar:
    if (artist not in artist_df["artist"].unique()) and (artist not in artist_df[0].unique()):
        i=i+1
        df = get_riaa(artist)
        artist_df = artist_df.append(df)
        if len(df) == 0:
            artist_df = artist_df.append([artist,None,None,None,None,None,None])
        print(i)
        if i % 5 == 0:
            with open('artist_df.pkl', 'wb') as f:
                pickle.dump(artist_df, f)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [315]:
print(len(song_df["artist"].unique()

3930

In [ ]:
artist_df["artist"].unique()